In [1]:
import numpy as np

In [2]:
#from IPython.display import Image
#Image(filename='table.png')

In [3]:
y = np.array([1160, 1200, 1280, 1450, 2000])

In [4]:
y.mean()

1418.0

Pseudo-residuals:

In [5]:
res = y - y.mean()
res

array([-258., -218., -138.,   32.,  582.])

In [6]:
def sum_of_errors(y):
    mu = y.mean()
    return ((y - mu)**2).sum()

## Finding the best stump to fit the residual
Since we have 5 points:

In [7]:
x =  np.array([750, 800, 850, 900, 950])

We consider 4 stumps at the following points:

In [8]:
[(x[i] + x[i+1])/2 for i in range(4)]

[775.0, 825.0, 875.0, 925.0]

In [9]:
res

array([-258., -218., -138.,   32.,  582.])

## Consider different splits

### Calculate sum of errors

In [10]:
i=1

In [11]:
res[:i], res[i:]

(array([-258.]), array([-218., -138.,   32.,  582.]))

In [12]:
sum_of_errors(res[:i]), sum_of_errors(res[i:])

(0.0, 389675.0)

In [13]:
(sum_of_errors(res[:i]) + sum_of_errors(res[i:]))/5

77935.0

In [14]:
for i in range(1, 5):
    print("Split "+ str(i)+":")
    print(res[:i], res[i:])
    print("MSE: "+ \
          str((sum_of_errors(res[:i]) + sum_of_errors(res[i:]))/5)+"\n") 
    
    
    

Split 1:
[-258.] [-218. -138.   32.  582.]
MSE: 77935.0

Split 2:
[-258. -218.] [-138.   32.  582.]
MSE: 56813.333333333336

Split 3:
[-258. -218. -138.] [ 32. 582.]
MSE: 31743.333333333332

Split 4:
[-258. -218. -138.   32.] [582.]
MSE: 9895.0



### Calculate avg of pseudo-residuals per leaf

In [15]:
i

4

In [16]:
np.mean(res[:i]), np.mean(res[i:])

(-145.5, 582.0)

## Create next stage's tree

In [17]:
T1 =np.array([-145.5, -145.5, -145.5, -145.5, 582])
T1

array([-145.5, -145.5, -145.5, -145.5,  582. ])

In [18]:
T1 = np.append(np.repeat(np.mean(res[:i]), len(res[:i])),\
               np.repeat(np.mean(res[i:]), len(res[i:])))
T1

array([-145.5, -145.5, -145.5, -145.5,  582. ])

## Add T1 to original f0

In [19]:
f1 = y.mean() +  T1
f1

array([1272.5, 1272.5, 1272.5, 1272.5, 2000. ])

In [20]:
y - f1

array([-112.5,  -72.5,    7.5,  177.5,    0. ])

# Regularizing Gradient Boosting
## Add T1 to original f0, scaled by learning rate

The hyperparameter $\nu$ is the "learning rate" should satisfy $\nu \in (0,1]$ (cf. $\eta$ in  gradient descent). Larger values indicate that the gradient boosting is minimizing the pseudo-residuals faster (i.e. with fewer stages) but also puts the model at risk of overfitting. Once we get to a point where the residuals in a leaf are 0, there's nothing left to learn. So then if we include that residual in another split, the next step will be trying to "unmemorize" that point. The balancing act of memorizing an unmemorizing is hard to understand– it's easier to understand gradual steps toward reducing the loss, so generally it's a better idea to just tune $\nu$ so that we can slowly keep learning points better and better without worrying about having to undo our learning. 

## Compare different learning rates

In [21]:
#learning rate
nu = 1

In [22]:
f1 = y.mean() + nu * T1
f1

array([1272.5, 1272.5, 1272.5, 1272.5, 2000. ])

In [23]:
y - f1

array([-112.5,  -72.5,    7.5,  177.5,    0. ])

We hit a leaf with 0 residual after just one tree. $(M = 1.)$ Danger of overfitting. Let's slow the learning rate down.

In [24]:
#learning rate
nu = 0.5

f1 = y.mean() + nu * T1
f1

array([1345.25, 1345.25, 1345.25, 1345.25, 1709.  ])

In [25]:
y - f1

array([-185.25, -145.25,  -65.25,  104.75,  291.  ])

Notice if we slow it down even further we will get even higher pseudo-residuals. That's fine if we keep building more and more trees $T_m$ to gradually reduce them. 

In [26]:
#learning rate
nu = 0.1
f1 = y.mean() + nu * T1
f1

array([1403.45, 1403.45, 1403.45, 1403.45, 1476.2 ])

In [27]:
res2 = y - f1
res2

array([-243.45, -203.45, -123.45,   46.55,  523.8 ])

#### Takeaway: Generally a smaller $\nu$ will necessitate a larger # of stages $M$. 

## Exercise:

1. Fit the next stage's regression tree $T_2$ using the residuals from $f_1$. 
2. What learning rate did you select? Why?
3. With your choice of learning rate, how many trees $M$ do you think you will need in order to achieve a good fit? How would you assess this quantitatively? 
4. Repeat the gradient boosting procedure but now using MAE instead of MSE. 

In [28]:
for i in range(1, 5):
    print("Split "+ str(i)+":")
    print(res2[:i], res2[i:])
    print("MSE: "+ \
          str((sum_of_errors(res2[:i]) + sum_of_errors(res2[i:]))/5)+"\n") 
    
    

Split 1:
[-243.45] [-203.45 -123.45   46.55  523.8 ]
MSE: 63669.634375

Split 2:
[-243.45 -203.45] [-123.45   46.55  523.8 ]
MSE: 45200.00833333334

Split 3:
[-243.45 -203.45 -123.45] [ 46.55 523.8 ]
MSE: 24270.089583333334

Split 4:
[-243.45 -203.45 -123.45   46.55] [523.8]
MSE: 9895.0



In [29]:
i=3

In [30]:
T2 = np.append(np.repeat(np.mean(res2[:i]), len(res2[:i])),\
               np.repeat(np.mean(res2[i:]), len(res2[i:])))
T2

array([-190.11666667, -190.11666667, -190.11666667,  285.175     ,
        285.175     ])

In [31]:
f2 = f1 + nu * T2

In [32]:
f2

array([1384.43833333, 1384.43833333, 1384.43833333, 1431.9675    ,
       1504.7175    ])

In [33]:
res3 = y - f2
res3

array([-224.43833333, -184.43833333, -104.43833333,   18.0325    ,
        495.2825    ])